## Decision Tree From Scratch

In [1]:
# Toy dataset with 3 columns, here last column is the label that we are trying to classify

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Red',2,'Grape']
]

In [2]:
# Columns
header = ["color", "diameter", "label"]

In [3]:
"""To check if column is numeric"""
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

Below class just records a 'column number' (e.g., 0 for Color) and a 'column value' (e.g., Green). The 'match' method is used to compare the feature value in an example to the feature value stored in the question.

In [4]:
class Question:
    """A Question is used to partition a dataset."""

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is a helper method to print the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [5]:
# Demo of the question class
Question(1, 3)

Is diameter >= 3?

In [6]:
def unique_vals(rows,col):
    """find unique values for a column in a dataset"""
    return set([row[col] for row in rows])

def class_counts(rows):
    counts={}
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

"""For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'."""
def partition(rows, question):
    true_rows, false_rows=[], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    counts=class_counts(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity-=prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainity):
    p=float(len(left))/(len(left) + len(right))
    return current_uncertainity - p* gini(left) - (1-p) * gini(right)

### Calling functions to demonstrate examples

In [7]:
unique_vals(training_data,0)

{'Green', 'Red', 'Yellow'}

In [8]:
class_counts(training_data)

{'Apple': 2, 'Grape': 3, 'Lemon': 1}

In [9]:
current_uncertainty = gini(training_data)
current_uncertainty

0.611111111111111

In [10]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.14444444444444443

In [11]:
def find_best_split(rows):
    best_gain=0
    best_question=None
    current_uncertainity=gini(rows)
    n_features=len(rows[0])-1
    
    for col in range(n_features):
        values=set([row[col]for row in rows])
        for val in values:
            question=Question(col,val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) ==0 or len(false_rows)==0:
                continue
            gain=info_gain(true_rows,false_rows,current_uncertainty)
            if gain >= best_gain:
                best_gain,best_question=gain, question
    return best_gain, best_question

In [12]:
best_gain, best_question = find_best_split(training_data)

In [13]:
best_question

Is diameter >= 3?

In [14]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [15]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [16]:
def build_tree(rows):
    """Builds the tree."""

    # Partitioing the dataset on each of the unique attribute,
    # calculating the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [17]:
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [18]:
my_tree = build_tree(training_data)

In [19]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Is diameter >= 2?
  --> True:
    Predict {'Grape': 1}
  --> False:
    Predict {'Grape': 2}


In [20]:
def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [21]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

### Testing and printing the test results

In [22]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [23]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
